# Functions to convert rasters or point clouds to other file formats (.xyz, .laz, .tif)

In [ ]:
import os
import glob
import laspy
import numpy as np
import rasterio
from rasterio.transform import from_origin


def laz2tiff (laz_file, tif_file, resolution=0.1):
    """
    Convert a .laz file to a GeoTIFF without interpolation, using a fixed 10cm resolution.

    :param laz_file: Path to the input .laz file.
    :param tif_file: Path to save the output GeoTIFF file.
    :param resolution: Fixed resolution in meters (default: 10cm).
    """
    # Read the .laz file using laspy
    with laspy.open(laz_file) as las:
        point_cloud = las.read()

    # Extract x, y, and z coordinates
    x = point_cloud.x
    y = point_cloud.y
    z = point_cloud.z

    # Define grid bounds
    min_x, max_x = np.min(x), np.max(x)
    min_y, max_y = np.min(y), np.max(y)

    # Define grid size
    num_x = int((max_x - min_x) / resolution) + 1
    num_y = int((max_y - min_y) / resolution) + 1

    # Create an empty raster grid with NaN values
    raster_grid = np.full((num_y, num_x), np.nan, dtype=np.float32)

    # Compute raster indices for each LiDAR point
    ix = ((x - min_x) / resolution).astype(int)
    iy = ((max_y - y) / resolution).astype(int)

    # Assign the last (or nearest) LiDAR height to each grid cell
    raster_grid[iy, ix] = z

    # Set transformation parameters
    transform = from_origin(min_x, max_y, resolution, resolution)

    # Write to GeoTIFF using Rasterio
    with rasterio.open(
        tif_file,
        "w",
        driver="GTiff",
        height=num_y,
        width=num_x,
        count=1,
        dtype=raster_grid.dtype,
        crs="EPSG:28992",  # Dutch Rijksdriehoekstelsel
        transform=transform,
        nodata=np.nan,
    ) as dst:
        dst.write(raster_grid, 1)


In [ ]:
import laspy
import os
import numpy as np

def laz2xyz(laz_file, output_folder):
    # Save input file name
    file_name = os.path.basename(laz_file)

    # Create output xyz-file
    output_xyz_file = os.path.splitext(file_name)[0] + ".xyz"
    output_xyz_path = os.path.join(output_folder, output_xyz_file)

    # Load .laz data
    with laspy.open(laz_file) as laz_reader:
        las = laz_reader.read()

        x = las.x
        y = las.y
        z = las.z
        scalar = las.classification
      

    # Stack into Nx3 array
    xyz_data = np.column_stack((x, y, z, scalar))

    # Save as .xyz (space-separated, 3 columns)
    np.savetxt(output_xyz_path, xyz_data, fmt="%.6f")

    print(f"saved as XYZ: {output_xyz_path}")
    


In [ ]:
import laspy
import os
import numpy as np

def xyz2laz(xyz_file, output_folder):
    #Save input file name
    file_name = os.path.basename(xyz_file)

    #Create output laz-file
    output_laz_file = os.path.splitext(file_name)[0] + ".laz"
    output_laz_path = os.path.join(output_folder, output_laz_file)

    
    # Load in the .xyz data
    data = np.loadtxt(xyz_file, usecols=(0, 1, 2))
    x, y, z, scalar = data[:, 0], data[:, 1], data[:, 2]

    #Create a LAS header
    header = laspy.LasHeader(point_format=1, version="1.2")
    header.scales = (0.01, 0.01, 0.01)  # Set scale factors
    header.offsets = (np.min(x), np.min(y), np.min(z))
    
    # Create LAS data
    las = laspy.LasData(header)
    las.x = x
    las.y = y
    las.z = z
    
    #Add the scalar field
    #las.add_extra_dim(laspy.ExtraBytesParams(name="scalar", type=np.float32))
    #las["scalar"] = scalar.astype(np.float32)

    #Save as laz
    with laspy.open(output_laz_path, mode="w", header=header) as las_writer:
       las_writer.write_points(las.points)

       print(f"saved as LAZ: {output_laz_path}")


In [ ]:
#Do it as a 
import os
#from rasterio.transform import from_origin
directory = ''  # set directory path
for entry in os.scandir(directory):  
    if entry.is_file():  
        xyz2laz(entry.path, output_folder = "")
